# Create Quaterly Frozen IC Universe 

In [1]:
import pandas as pd
import polars as pl
import numpy as np

In [3]:
TGTQTR = '2024Q4'
CUR_PROC_WK = '20240812'
CUR_WK = '20240726' #*xponent data week, even in month closing week, still use FRIDAY date;
FILENAME = 'Q424_Exclusion_List_formatted'
BASE = 'MASTER_PROFILE_2024Q4'

ICQ = f's3://vortex-staging-a65ced90/PYADM/quaterly/{TGTQTR}/reference/'

exclusions - 

In [ ]:
# Importing Raw FILE - 
exclusion = (
    pl.from_pandas(
        pd.read_excel(
            f's3://vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/quaterly_raw/{FILENAME}.xlsx',
            sheet_name= 'exclusion'
        )
    ).unique()
)

ukaddr = (
    pl.from_pandas(
        pd.read_excel(
            f's3://vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/quaterly_raw/{FILENAME}.xlsx',
            sheet_name= 'ukaddr'
        )
    ).unique()
)

exclusion = exclusion['IID'].unique().to_list()
ukaddr = ukaddr['IID'].unique().to_list()

In [45]:
CUSTOMER_MASTER_IC_LIN = (
    pl.read_parquet(f'{ICQ}{BASE}.parquet')
    .filter(
        (~pl.col('IID').is_in(exclusion)) & (~pl.col('IID').is_in(ukaddr))
    )
    .filter(
        (pl.col('SPEC_INCL_LIN')=='Y') & (pl.col('CustomerStatusCode')=='Active') & (pl.col('MatchCode')!='01')
    )
)

In [49]:
# Export
(
    CUSTOMER_MASTER_IC_LIN
    .to_pandas()
    .to_parquet(f'{ICQ}CUSTOMER_MASTER_IC_LIN.parquet',index = False)
)